# Autoregressive Model: AR(1)

$$y_{t}=c+ \phi y_{t-1} + \epsilon_{t}, \quad \epsilon_{t} \overset{i.i.d.} \sim N(0,\sigma^2)$$

In [17]:
import numpy as np

def fit_ar1(y):
    y = np.asarray(y, dtype = float)
    y_t = y[1:]
    y_lag = y[:-1]
    T = y_t.shape[0]
    X = np.column_stack([np.ones(T),y_lag])
    XtX = X.T @ X
    Xty = X.T @ y_t
    beta = np.linalg.solve(XtX,Xty)
    c_hat , phi_hat = beta
    eps = y_t - X @ beta
    k = X.shape[1]
    sigma2_hat = (eps @ eps) /(T-k)
    return {
        "c": c_hat,
        "phi": phi_hat,
        "sigma2": sigma2_hat
    }
    

In [18]:
np.random.seed(0)
T = 500
c_true = 0.5
phi_true = 0.8
sigma_true = 1.0

y = np.zeros(T)
for t in range(1, T):
    y[t] = c_true + phi_true * y[t-1] + sigma_true * np.random.randn()

res = fit_ar1(y)
print("c_hat:",  res["c"])
print("phi_hat:", res["phi"])
print("sigma^2_hat:", res["sigma2"])

c_hat: 0.4349959770117204
phi_hat: 0.8169098075200518
sigma^2_hat: 1.0014549985559429


# Moving Average Model: MA(1)

$$y_{t} = \epsilon_{t} + \theta \epsilon_{t-1}, \quad \epsilon_{t} \overset{i.i.d}\sim N(0,\sigma^2)$$

$$SSE(\theta)= \sum_{t=1}^{T} \epsilon_{t}(\theta)^2$$

In [19]:
import numpy as np

def numerical_grad(f, x, eps=1e-5):
    x = np.asarray(x, dtype=float)
    g = np.zeros_like(x)
    for i in range(len(x)):
        x_step = x.copy()
        x_step[i] += eps
        f_plus = f(x_step)
        x_step[i] -= 2 * eps
        f_minus = f(x_step)
        g[i] = (f_plus - f_minus) / (2 * eps)
    return g

def gradient_descent(f, x0, lr=1e-3, max_iter=10000, tol=1e-6):
    x = np.asarray(x0, dtype=float)
    fx = f(x)
    for it in range(max_iter):
        g = numerical_grad(f, x)
        if np.linalg.norm(g) < tol:
            break
        x_new = x - lr * g
        fx_new = f(x_new)
        if fx_new > fx:
            lr *= 0.5
        else:
            x = x_new
            fx = fx_new
    return x, fx

In [24]:
def fit_ma1_gd(y):
    y = np.asarray(y, dtype = float)
    T = y.shape[0]
    def css_sse(params):
        mu, theta = params
        eps_prev = 0.0
        sse = 0.0
        for t in range(T):
            eps_t = y[t] - mu - theta * eps_prev
            sse += eps_t * eps_t
            eps_prev = eps_t
        return sse
    mu0 = y.mean()
    theta0 = 0.0
    x0 = np.array([mu0, theta0], dtype=float)
    params_opt, sse_min = gradient_descent(css_sse, x0, lr=1e-3)
    mu_hat, theta_hat = params_opt
    sigma2_hat = sse_min /T
    return {
        "mu": mu_hat,
        "theta": theta_hat,
        "sigma2": sigma2_hat
    }

In [25]:
if __name__ == "__main__":
    np.random.seed(0)

    T = 500
    mu_true = 1.0
    theta_true = 0.6
    sigma_true = 1.0

    eps = np.zeros(T + 1)
    y = np.zeros(T)
    for t in range(T):
        eps[t+1] = sigma_true * np.random.randn()
        y[t] = mu_true + eps[t+1] + theta_true * eps[t]

    res = fit_ma1_gd(y)
    print("True mu      :", mu_true)
    print("Estimated mu :", res["mu"])
    print("True theta   :", theta_true)
    print("Estimated theta:", res["theta"])
    print("True sigma^2 :", sigma_true**2)
    print("Estimated sigma^2:", res["sigma2"])

True mu      : 1.0
Estimated mu : 0.962785228661364
True theta   : 0.6
Estimated theta: 0.5919857830747333
True sigma^2 : 1.0
Estimated sigma^2: 0.99633536394758


# Autoregressive Moving Average Model: ARMA(1,1)

$$y_{t} - \mu = \phi(y_{t-1}-\mu)+\epsilon_{t}+\theta \epsilon_{t-1}, \quad \epsilon_{t} \overset{i.i.d.}\sim N(0,\sigma^2)$$

$$\epsilon_{t}= (y_{t} - \mu) - \phi(y_{t-1}-\mu) - \theta \epsilon_{t-1}$$

$$CSS(\mu, \phi, \theta)= \sum_{t=1}^{T-1}\epsilon_{t}^2$$

In [28]:
def fit_arma11(y, lr=1e-3, max_iter=5000):
    y = np.asarray(y, dtype=float)
    def css_sse(params):
        mu, phi, theta = params
        z = y - mu      
        eps_prev = 0.0
        sse = 0.0
        for t in range(1, T):
            eps_t = z[t] - phi * z[t-1] - theta * eps_prev
            sse += eps_t * eps_t
            eps_prev = eps_t
        return sse
    mu0 = y.mean()
    x0 = np.array([mu0, 0.0, 0.0])
    params_opt, sse_min = gradient_descent(css_sse, x0, lr=lr, max_iter=max_iter)
    mu_hat, phi_hat, theta_hat = params_opt
    sigma2_hat = sse_min / (T - 1)
    return {
        "mu": mu_hat,
        "phi": phi_hat,
        "theta": theta_hat,
        "sigma2": sigma2_hat
    }

In [29]:
if __name__ == "__main__":
    np.random.seed(0)
    T = 500
    mu_true = 1.0
    phi_true = 0.7
    theta_true = 0.4
    sigma_true = 1.0

    y = np.zeros(T)
    eps = np.zeros(T+1)   
    y[0] = mu_true + sigma_true * np.random.randn()

    for t in range(1, T):
        eps[t] = sigma_true * np.random.randn()
        y[t] = (mu_true
                + phi_true * (y[t-1] - mu_true)
                + eps[t]
                + theta_true * eps[t-1])

    res = fit_arma11(y)
    print("True mu    :", mu_true)
    print("Est  mu    :", res["mu"])
    print("True phi   :", phi_true)
    print("Est  phi   :", res["phi"])
    print("True theta :", theta_true)
    print("Est  theta :", res["theta"])
    print("True sigma^2 :", sigma_true**2)
    print("Est  sigma^2:", res["sigma2"])

True mu    : 1.0
Est  mu    : 0.8630091574086183
True phi   : 0.7
Est  phi   : 0.7386729400527929
True theta : 0.4
Est  theta : 0.3503720704312446
True sigma^2 : 1.0
Est  sigma^2: 0.9891295299918458


/var/folders/_b/28jfj19121qf32rdzd8twc000000gn/T/ipykernel_32194/1914848545.py:10: RuntimeWarning: overflow encountered in scalar multiply
  sse += eps_t * eps_t
